In [9]:
!pip install pandas transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from QA_model.qa import (
    load_csv_data_from_dirs,
    init_model,
    build_rag_data,
    rag_search,
    answer_question
)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_dir = "/data/jung/src/datas"  # CSV들이 들어있는 상위 폴더
df_all = load_csv_data_from_dirs(base_dir=base_dir)   # 모든 CSV 통합 로드
print(f"로딩된 CSV 행 개수: {len(df_all)}")


rag_data = build_rag_data(df_all) #아직 임베딩 벡터가 없으므로 임의로 모든 항목을 리스트에 저장해서 해당 리스트를 db 처럼 사용중
print(f"RAG 데이터 개수: {len(rag_data)}")

로딩된 CSV 행 개수: 1240
RAG 데이터 개수: 1240


In [3]:
# Qwen 사용시 질의 끝 혹은 코드 프롬프트에 한국어로 설명해달라는 프롬프트를 추가해야 함 아니면 중국어로만 나옴..
# model_handle = init_model("Qwen/Qwen2.5-7B-Instruct")
# 또는
model_handle = init_model("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")
# 또는 (API 호출식)
# model_handle = init_model("GPT-4o-mini")   #GPT 이용 시에는 QA_model 폴더에 .env 파일 생성해야 함

Loading EXAONE model: LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct


Loading checkpoint shards: 100%|██████████| 7/7 [00:09<00:00,  1.31s/it]


In [4]:
question_text = "카카오는 언제 비대면 주택담보대출을 출시했나요? 정답은 한국어로 대답해줘" 
answer = answer_question(question_text, rag_data, model_handle, top_k=3, use_bm25=False)
print(answer)


카카오뱅크는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다.


### validation 과 연결

#### csv 파일에 있는 쿼리에 대해 모두 응답하면서 점수 측정 후 저장하는 구조 

In [5]:
from evaluation.GPT_evaluation import G_evaluate
import pandas as pd
from QA_model.qa import bm25_rag_search


df = pd.read_csv("/data/jung/src/qa_validation_dataset_4_cleaning.csv")

query_list = df.apply(lambda row : [row["question"], row["answer"]], axis = 1).to_list()

relevant_items = rag_search(
        question=query_list[0][0],
        data_list=rag_data,
        top_k=3,
        use_bm25=False
        )

context_parts = []
for idx, item in enumerate(relevant_items, start=1):
    if item["summary"]:
        context_parts.append(f"{idx}. {item['summary']}")
    elif item["original_content"]:
        context_parts.append(f"{idx}. {item['original_content']}")
retrieved_contexts = "\n".join(context_parts)

print(retrieved_contexts)

generated_answer = answer_question(query_list[0][0], rag_data, model_handle, top_k=3, use_bm25=False)

eval_score = G_evaluate(query_list[0][0], retrieved_contexts, query_list[0][1], generated_answer)

print(eval_score)

1. SK하이닉스는 향후 3개년간 주주환원 정책을 공시하며, 연간 고정배당금을 1,500원으로 상향하고,  Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다고 밝혔다. 이는 메모리 업황 특성상 변동성이 큰 상황에서 내린 합리적인 결정이라고 판단된다.
2. SK하이닉스의 2025~2027년 주주환원 정책에 대한 기준과 변경 사항을 정리한 테이블로, 배당금 및 Free Cash Flow 활용 방안이 포함되어 있습니다.
3. SK하이닉스의 주요 재무 지표와 주식 관련 데이터를 나타내는 테이블로, KOSPI 지수, 시가총액, 외국인 지분율 등 다양한 정보를 포함하고 있습니다.
#### 1. Retrieval Evaluation (20 points):
1. **Do any of the retrieved contexts show strong similarity to the Ground Truth?** (5 points)  
   [Yes] - Justification: The first retrieved context closely mirrors the Ground Truth, mentioning the increase in fixed dividends to 1,500 won and the prioritization of Free Cash Flow for financial stability.

2. **Do the retrieved contexts collectively capture essential information from the Ground Truth?** (5 points)  
   [Yes] - Justification: The first context captures the key changes in the shareholder return policy, while the second context provides a structured overview of the policy, ensuring essential informati

#### 추가 => 모델 클래스 분류 후 사용

In [1]:
from QA_model.qwen_model import QwenModel
from QA_model.exaone_model import ExaoneModel
from QA_model.gpt_model import GPTModel
from QA_model.utils import load_csv_data_from_dirs, build_rag_data

# 데이터 로드
base_dir = "/data/jung/src/datas"
df_all = load_csv_data_from_dirs(base_dir)
rag_data = build_rag_data(df_all)

# 모델 초기화 => 세개 중 선택
# model1 = QwenModel()
model2 = ExaoneModel()
# model3 = GPTModel(model_name="gpt-4o-mini")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 질문 및 답변
retrieval_results = "카카오뱅크는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다."
question = "카카오는 언제 비대면 주택담보대출을 출시했나요?"

prompt = f"주어진 문서를 반드시 참고하여 질문에 대한 답을 하세요.\n\n주어진 문서 : {retrieval_results}\n질문 : {question}"
# print(model1.answering(prompt))
# print(model2.answering(prompt))
print(model2.answering(prompt))

카카오는 2022년 2월에 비대면 주택담보대출 상품을 출시했습니다.


In [4]:
from DB.chromadb_storing import ChromaDB
from langchain.schema import Document
from utils import get_only_paragraphs, get_all_datas, create_documents
BASE_DIR = '/data/jung/src/datas' # 데이터가 저장돼 있는 루트를 의미합니다.

df = get_only_paragraphs(BASE_DIR)
# df = get_all_datas(BASE_DIR) # 이미지와 테이블이 포함된 전체 데이터도 가져올 수 있습니다.
df[:3]


,id,type,image_route,dir_route,file_name,page,investment,company_name,summary,table,original_content
2,7,paragraph,NaN,datas/네이버_교보증권(2024.11.11),네이버_교보증권(2024.11.11).pdf,1,교보증권(2024.11.11),네이버,"교보증권은 네이버의 3분기 실적 발표에서 매출 2.72조 원(YoY +11.1%),...",NaN,<br><p id='7' data-category='paragraph' style=...
5,11,paragraph,NaN,datas/네이버_교보증권(2024.11.11),네이버_교보증권(2024.11.11).pdf,1,교보증권(2024.11.11),네이버,"영업비용은 2.19조원으로 전년대비 6.1% 증가했고, OPM은 19.3%를 기록했...",NaN,<br><p id='11' data-category='paragraph' style...
6,13,paragraph,NaN,datas/네이버_교보증권(2024.11.11),네이버_교보증권(2024.11.11).pdf,1,교보증권(2024.11.11),네이버,"네이버는 개인화 앱 개편 영향으로 서치 플랫폼 성장률이 가속화되고 있으며, 자체 경...",NaN,<br><p id='13' data-category='paragraph' style...


In [5]:
documents = create_documents(df)
documents[:3]

[Document(metadata={'id': 7, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/네이버_교보증권(2024.11.11)', 'file_name': '네이버_교보증권(2024.11.11).pdf', 'page': 1, 'investment': '교보증권(2024.11.11)', 'company_name': '네이버', 'table': nan, 'original_content': "<br><p id='7' data-category='paragraph' style='font-size:16px'>매출 2.72조원(YoY +11.1%)로 컨센서스(2.62조원) 부합, 영익은 5,253억원(+38.2%) 기록<br>해 시장예상치(4,865억원) 상회. 서치플랫폼(YoY +11.0%) 중 DA는 피드 지면 확대 및 타<br>게팅 고도화로 +11.0%, SA는 플레이스광고 성장, 상품 개선 및 외부 매체 상품 확대로<br>+9.5% 성장. 커머스 매출(YoY +12.0%)은 중개/판매 중심 성장. 도착보장/브랜드솔루션<br>사용률 증가와 브랜드스토어 거래액 성장으로 On-Platform 거래액(+10.0%) 증가보다 중<br>개/판매 매출 크게(+24.6%) 증가. 핀테크 결제액은 외부결제(+37.7%), 주문/예약 및 현장<br>결제 확대 따른 오프라인(YoY +78%) 성장으로 18.6조원(+22.1%) 달성, 핀테크 매출<br>+13.0% 성장. 컨텐츠 매출은 유료 컨텐츠 및 광고 매출 증대로 웹툰 매출 4,240억원<br>(+11.6%, 동일환율 기준 +13.5%) 기록한 가운데 스노우(-46.5%, 네이버제트 제외 영향<br>제거시 YoY +9.0%)에서 네이버제트 연결 제외되며 +6.4%. 클라우드 매출은 라인웍스 유<br>료 ID수 확대, 뉴로클라우드 및 사우디 주택부 디지털트윈 매출 발생으로 YoY +17.0%.</p>"}, page_

In [6]:
# add_docs를 통해 문서들을 저장합니다.

collection_name = 'chrdb.db'
persist_directory = '/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/data/db'
DB = ChromaDB(collection_name, persist_directory)
DB.create_and_add(documents)


Received 388 documents for embedding.
Adding batch 1 of 4...


/data/level4-nlp-finalproject-hackathon-nlp-09-lv3/src/DB/chromadb_storing.py:34: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


Adding batch 2 of 4...
Adding batch 3 of 4...
Adding batch 4 of 4...
Collection 'chrdb.db' created and persisted at '/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/data/db'.


/data/level4-nlp-finalproject-hackathon-nlp-09-lv3/src/DB/chromadb_storing.py:49: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.db.persist()


In [7]:
db = DB.verify_db()
DB.load_collection()
db = DB.verify_db()

Total documents in collection 'chrdb.db': 1164
Collection 'chrdb.db' loaded from '/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/data/db'.
Total documents in collection 'chrdb.db': 1164


In [8]:
import sqlite3

def inspect_sqlite_db(db_path):
    """
    Inspect the structure and contents of an SQLite3 database.
    
    :param db_path: Path to the SQLite3 database file.
    """
    try:
        # 데이터베이스 연결
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 모든 테이블 이름 가져오기
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(f"- {table[0]}")

        # 각 테이블의 스키마와 데이터를 확인
        for table in tables:
            table_name = table[0]
            print(f"\nSchema for table '{table_name}':")
            cursor.execute(f"PRAGMA table_info({table_name});")
            schema = cursor.fetchall()
            for column in schema:
                print(column)

            print(f"\nData in table '{table_name}':")
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
            rows = cursor.fetchall()
            for row in rows:
                print(row)

        # 연결 종료
        conn.close()
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        

In [9]:
db_path = '/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/data/db/chroma.sqlite3'  # SQLite3 파일 경로
inspect_sqlite_db(db_path)

Tables in the database:
- migrations
- embeddings_queue
- embeddings_queue_config
- collection_metadata
- segment_metadata
- tenants
- databases
- collections
- maintenance_log
- segments
- embeddings
- embedding_metadata
- max_seq_id
- embedding_fulltext_search
- embedding_fulltext_search_data
- embedding_fulltext_search_idx
- embedding_fulltext_search_content
- embedding_fulltext_search_docsize
- embedding_fulltext_search_config

Schema for table 'migrations':
(0, 'dir', 'TEXT', 1, None, 1)
(1, 'version', 'INTEGER', 1, None, 2)
(2, 'filename', 'TEXT', 1, None, 0)
(3, 'sql', 'TEXT', 1, None, 0)
(4, 'hash', 'TEXT', 1, None, 0)

Data in table 'migrations':
('embeddings_queue', 1, '00001-embeddings.sqlite.sql', 'CREATE TABLE embeddings_queue (\n    seq_id INTEGER PRIMARY KEY,\n    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,\n    operation INTEGER NOT NULL,\n    topic TEXT NOT NULL,\n    id TEXT NOT NULL,\n    vector BLOB,\n    encoding TEXT,\n    metadata TEXT\n);\n', 'd3755

In [13]:
import pandas as pd
import time
from evaluation.GPT_evaluation import G_evaluate
from retrievals import bm25, dpr, ensemble
from QA_model.qwen_model import QwenModel
from QA_model.exaone_model import ExaoneModel
from QA_model.gpt_model import GPTModel
from DB.chromadb_storing import ChromaDB
from langchain.schema import Document
from utils import get_only_paragraphs, get_all_datas, create_documents
import re

def extract_scores(eval_string):
    retrieval_score_match = re.search(r"Total Retrieval Score\D+(\d+)", eval_string)
    total_score_match = re.search(r"Total Score\D+(\d+)", eval_string)

    retrieval_score = int(retrieval_score_match.group(1)) if retrieval_score_match else None
    total_score = int(total_score_match.group(1)) if total_score_match else None

    return retrieval_score, total_score


# Retriever 초기화 => DB 는 생성 안되어서 이건 되는 사람이 테스트좀..
topk = 3
DPRRetriever = dpr(db, topk=topk)
BM25Retriever = bm25(documents, topk=topk)
retrievals = [DPRRetriever, BM25Retriever]
weights = [0.5, 0.5]
search_type = 'mmr'
ensemble_retriever = ensemble(retrievals, topk=topk, weights=weights, search_type=search_type)

retrievers = {
    "BM25": BM25Retriever,
    "DPR": DPRRetriever,
    "Ensemble": ensemble_retriever
}

# 모델 초기화
models = {
    "QwenModel": QwenModel(),
    # "ExaoneModel": ExaoneModel(),
    # "GPTModel": GPTModel(model_name="gpt-4o-mini")
}

# 평가 데이터 로드
validation_df = pd.read_csv("/data/jung/src/qa_validation_dataset_4_cleaning.csv")
query_list = validation_df.apply(lambda row: [row["question"], row["answer"]], axis=1).to_list()

# 결과 저장을 위한 리스트
results = []

for model_name, model in models.items():
    for retriever_name, retriever in retrievers.items():
        for question, ground_truth in query_list:
            
            start_time = time.time()
            
            # Retrieval
            retrieved_docs = retriever.invoke(question)
            retrieved_texts = [doc.page_content for doc in retrieved_docs]  # Retrieved 텍스트만 사용
            retrieved_contexts = "\n".join(retrieved_texts)

            # 정답 생성
            prompt = f"주어진 문서를 반드시 참고하여 질문에 대한 답을 하세요.\n\n주어진 문서 : {retrieved_contexts}\n질문 : {question}"
            generated_answer = model.answering(prompt)

            # 평가
            eval_score = G_evaluate(question, retrieved_contexts, ground_truth, generated_answer)
            
            retrieval_score, total_score = extract_scores(eval_score)
            
            end_time = time.time()
            inference_time = end_time - start_time

            # 결과 저장
            results.append({
                "Model": model_name,
                "Question": question,
                "Retriever_name" : retriever_name
                "Total Score": total_score,
                "Inference Time (s)": round(inference_time, 2)
            })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과 CSV 파일 저장
results_df.to_csv("/data/jung/src/evaluation_results_qwen.csv", index=False, encoding="utf-8-sig")

print("Evaluation completed. Results saved to '/data/jung/src/evaluation_results'.")


Loading checkpoint shards: 100%|██████████| 7/7 [00:09<00:00,  1.35s/it]


Evaluation completed. Results saved to '/data/jung/src/evaluation_results'.
